In [8]:
# -*- coding: utf-8 -*-

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader


from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

In [6]:
import os
import json

list_path_dir = [
    "./Data/Ngành đào tạo",
    "./Data/test",
    "./Data/Phòng đào tạo",
    "./Data/Điểm chuẩn PTIT năm 2023",
    "./Data/Thông tin tuyển sinh",
    "./Data/Others",
    './Data/gen17052024',
    './Data/gen21052024',
]

list_dict_intent = [
    {"train_dataset_tt_1": "Chỉ tiêu tuyển sinh"},
    {"train_dataset_tt_2": "Câu hỏi xét tuyển"},
    {"train_dataset_tt_3": "Học phí"},
    {"train_dataset_tt_4": "Thông tin liên hệ"},
    {"train_dataset_tt_5": "Phương thức 1"},
    {"train_dataset_tt_6": "Phương thức 2"},
    {"train_dataset_tt_7": "Phương thức 3"},
    {"train_dataset_tt_8": "Phương thức 4"},
    {"train_dataset_tt_9": "Quy đổi điểm chứng chỉ quốc tế"},
    {"train_dataset_tt_10": "Phương thức xét tuyển"},
    {"train_dataset_tt_11": "Đối tượng tuyển sinh"},
    {"train_dataset_tt_12": "Thời gian xét tuyển và công bố kết quả"},
    {"train_dataset_tt_13": "Thời gian nộp hồ sơ đăng ký xét tuyển kết hợp"},
    {
        "train_dataset_tt_14": "Thời gian nộp hồ sơ đăng ký xét tuyển đánh giá năng lực-tư duy"
    },
    {"train_dataset_tt_15": "Tổ hợp xét tuyển"},
    {"train_dataset_tt_16": "Xét tuyển học bạ"},
    {"train_dataset_test_0": "Ngành mới"},
    {"train_dataset_test_1": "Ngưỡng đảm bảo"},
    {"train_dataset_test_2": "Phương thức tuyển sinh"},
    {"train_dataset_test_3": "Ngưỡng đảm bảo đánh giá năng lực-tư duy"},
    {"train_dataset_test_4": "Hồ sơ xét tuyển hợp lệ"},
    {"train_dataset_test_5": "Thời gian đăng ký-điều chỉnh nguyện vọng-nộp kinh phí"},
    {"train_dataset_test_6": "Địa chỉ nộp hồ sơ xét tuyển"},
    {"./gen17052024/Câu hỏi": "Câu hỏi xét tuyển"},
    {"./gen17052024/Giải hsg": "Giải học sinh giỏi"},
    {
        "./gen17052024/Kết thúc đăng ký xét tuyển sớm": "Thời gian kết thúc đăng ký xét tuyển sớm"
    },
    {"./gen17052024/Xét học bạ": "Xét tuyển học bạ"},
    {"./gen17052024/Đăng ký xét tuyển": "Đăng ký xét tuyển"},
    {"./gen21052024/Chính sách học bổng": "chinh_sach_hoc_bong"},
    {"./gen21052024/Công thức tính điểm ưu tiên": "cong_thuc_tinh_diem_uu_tien"},
    {"./gen21052024/Cẩm nang": "cam_nang"},
    {"./gen21052024/Danh sách các ngành": "danh_sach_nganh"},
    {
        "./gen21052024/Thời gian đăng kí và điều chỉnh nguyện vọng": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong"
    },
    {"./gen21052024/Điểm_cộng_đối_tượng_ưu_tiên": "diem_cong_doi_tuong_uu_tien"},
]

list_files_ = [
    {"./Thông tin tuyển sinh/Chỉ tiêu tuyển sinh.txt": "chi_tieu_tuyen_sinh"},
    {"./Thông tin tuyển sinh/Câu hỏi.txt": "cau_hoi_xet_tuyen"},
    {"./Thông tin tuyển sinh/Học phí.txt": "hoc_phi"},
    {"./Thông tin tuyển sinh/Liên hệ và hỗ trợ.txt": "thong_tin_lien_he"},
    {"./Thông tin tuyển sinh/Phương thức 1.txt": "phuong_thuc_1"},
    {"./Thông tin tuyển sinh/Phương thức 2.txt": "phuong_thuc_2"},
    {"./Thông tin tuyển sinh/Phương thức 3.txt": "phuong_thuc_3"},
    {"./Thông tin tuyển sinh/Phương thức 4.txt": "phuong_thuc_4"},
    {
        "./Thông tin tuyển sinh/Quy đổi chứng chỉ, giải.txt": "quy_doi_diem_chung_chi_quoc_te"
    },
    {
        "./Thông tin tuyển sinh/Thông báo mở đăng ký xét tuyển.txt": "phuong_thuc_xet_tuyen"
    },
    {"./Thông tin tuyển sinh/Thông tin tuyển sinh.txt": "doi_tuong_tuyen_sinh"},
    {
        "./Thông tin tuyển sinh/Thời gian đăng kí.txt": "thoi_gian_xet_tuyen_va_cong_bo_ket_qua"
    },
    {
        "./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_kết_hợp_XTKH.txt": "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_ket_hop"
    },
    {
        "./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_ĐGNL,_ĐGTD.txt": "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_danh_gia_nang_luc-tu_duy"
    },
    {"./Thông tin tuyển sinh/Tổ hợp xét tuyển.txt": "to_hop_xet_tuyen"},
    {"./Thông tin tuyển sinh/Xét học bạ.txt": "xet_tuyen_hoc_ba"},
    {"./test/Các ngành mới.txt": "nganh_moi"},
    {"./test/Các ngưỡng đảm bảo đầu vào.txt": "nguong_dam_bao"},
    {"./test/Các phương thức xét tuyển.txt": "phuong_thuc_tuyen_sinh"},
    {"./test/DgnlDgtd.txt": "nguong_dam_bao_danh_gia_nang_luc-tu_duy"},
    {"./test/Hồ sơ hợp lệ.txt": "ho_so_xet_tuyen_hop_le"},
    {
        "./test/Thời gian đăng kí và điều chỉnh nguyện vọng.txt": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong-nop_kinh_phi"
    },
    {"./test/Địa chỉ nộp hồ sơ xét tuyển.txt": "dia_chi_nop_ho_so_xet_tuyen"},
    {"./gen17052024/Câu hỏi.txt": "cau_hoi_xet_tuyen"},
    {"./gen17052024/Giải hsg.txt": "giai_hoc_sinh_gioi"},
    {
        "./gen17052024/Kết thúc đăng ký xét tuyển sớm.txt": "thoi_gian_ket_thuc_dang_ky_xet_tuyen_som"
    },
    {"./gen17052024/Xét học bạ.txt": "xet_tuyen_hoc_ba"},
    {"./gen17052024/Đăng ký xét tuyển.txt": "dang_ky_xet_tuyen"},
    {"./gen21052024/Chính sách học bổng.txt": "chinh_sach_hoc_bong"},
    {"./gen21052024/Công thức tính điểm ưu tiên.txt": "cong_thuc_tinh_diem_uu_tien"},
    {"./gen21052024/Cẩm nang.txt": "cam_nang"},
    {"./gen21052024/Danh sách các ngành.txt": "danh_sach_nganh"},
    {
        "./gen21052024/Thời gian đăng kí và điều chỉnh nguyện vọng.txt": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong"
    },
    {"./gen21052024/Điểm_cộng_đối_tượng_ưu_tiên.txt": "diem_cong_doi_tuong_uu_tien"},
]

dict_data = {}

for i in list_dict_intent:
    for key, value in i.items():
        with open(f"./Data/{key}.json", "r", encoding="utf-8") as f:
            data = json.load(f)
            # print(value)
            if value not in dict_data:
                dict_data[value] = []
            for k, q in data["queries"].items():
                dict_data[value].append(q)
dict_data

{'Chỉ tiêu tuyển sinh': ['Tổng số chỉ tiêu tuyển sinh của Học viện Bưu chính Viễn thông năm 2024 là bao nhiêu?',
  'Có bao nhiêu chỉ tiêu tuyển sinh tại cơ sở phía Bắc của Học viện Bưu chính Viễn thông?',
  'Chỉ tiêu tuyển sinh ngành Kỹ thuật Điện tử viễn thông tại cơ sở phía Nam là bao nhiêu?',
  'Có bao nhiêu chỉ tiêu tuyển sinh ngành Công nghệ Kỹ thuật Điện-điện tử tại cơ sở phía Bắc?',
  'Chỉ tiêu tuyển sinh ngành An toàn thông tin tại cơ sở phía Nam là bao nhiêu?',
  'Có bao nhiêu chỉ tiêu tuyển sinh ngành Khoa học máy tính (định hướng Khoa học dữ liệu) tại cơ sở phía Bắc?',
  'Chỉ tiêu tuyển sinh ngành Mạng máy tính và truyền thông dữ liệu tại cơ sở phía Bắc là bao nhiêu?',
  'Có bao nhiêu chỉ tiêu tuyển sinh ngành Cử nhân Công nghệ thông tin (định hướng ứng dụng) tại cơ sở phía Bắc?',
  'Chỉ tiêu tuyển sinh ngành Thương mại điện tử tại cơ sở phía Bắc là bao nhiêu?',
  'Có bao nhiêu chỉ tiêu tuyển sinh ngành Marketing tại cơ sở phía Nam?',
  'Chỉ tiêu tuyển sinh ngành Kế toán tại

# load key

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-tZ5Uxrp7mtytZGbGRAMWT3BlbkFJIMLK05g3ZOlYdz8PIHkK"

In [9]:
list_files_ = [
    {"./Thông tin tuyển sinh/Chỉ tiêu tuyển sinh.txt": "chi_tieu_tuyen_sinh"},
    {"./Thông tin tuyển sinh/Câu hỏi.txt": "cau_hoi_xet_tuyen"},
    {"./Thông tin tuyển sinh/Học phí.txt": "hoc_phi"},
    {"./Thông tin tuyển sinh/Liên hệ và hỗ trợ.txt": "thong_tin_lien_he"},
    {"./Thông tin tuyển sinh/Phương thức 1.txt": "phuong_thuc_1"},
    {"./Thông tin tuyển sinh/Phương thức 2.txt": "phuong_thuc_2"},
    {"./Thông tin tuyển sinh/Phương thức 3.txt": "phuong_thuc_3"},
    {"./Thông tin tuyển sinh/Phương thức 4.txt": "phuong_thuc_4"},
    {
        "./Thông tin tuyển sinh/Quy đổi chứng chỉ, giải.txt": "quy_doi_diem_chung_chi_quoc_te"
    },
    {
        "./Thông tin tuyển sinh/Thông báo mở đăng ký xét tuyển.txt": "phuong_thuc_xet_tuyen"
    },
    {"./Thông tin tuyển sinh/Thông tin tuyển sinh.txt": "doi_tuong_tuyen_sinh"},
    {
        "./Thông tin tuyển sinh/Thời gian đăng kí.txt": "thoi_gian_xet_tuyen_va_cong_bo_ket_qua"
    },
    {
        "./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_kết_hợp_XTKH.txt": "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_ket_hop"
    },
    {
        "./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_ĐGNL,_ĐGTD.txt": "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_danh_gia_nang_luc-tu_duy"
    },
    {"./Thông tin tuyển sinh/Tổ hợp xét tuyển.txt": "to_hop_xet_tuyen"},
    {"./Thông tin tuyển sinh/Xét học bạ.txt": "xet_tuyen_hoc_ba"},
    {"./test/Các ngành mới.txt": "nganh_moi"},
    {"./test/Các ngưỡng đảm bảo đầu vào.txt": "nguong_dam_bao"},
    {"./test/Các phương thức xét tuyển.txt": "phuong_thuc_tuyen_sinh"},
    {"./test/DgnlDgtd.txt": "nguong_dam_bao_danh_gia_nang_luc-tu_duy"},
    {"./test/Hồ sơ hợp lệ.txt": "ho_so_xet_tuyen_hop_le"},
    {
        "./test/Thời gian đăng kí và điều chỉnh nguyện vọng.txt": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong-nop_kinh_phi"
    },
    {"./test/Địa chỉ nộp hồ sơ xét tuyển.txt": "dia_chi_nop_ho_so_xet_tuyen"},
    {"./gen17052024/Câu hỏi.txt": "cau_hoi_xet_tuyen"},
    {"./gen17052024/Giải hsg.txt": "giai_hoc_sinh_gioi"},
    {
        "./gen17052024/Kết thúc đăng ký xét tuyển sớm.txt": "thoi_gian_ket_thuc_dang_ky_xet_tuyen_som"
    },
    {"./gen17052024/Xét học bạ.txt": "xet_tuyen_hoc_ba"},
    {"./gen17052024/Đăng ký xét tuyển.txt": "dang_ky_xet_tuyen"},
    {"./gen21052024/Chính sách học bổng.txt": "chinh_sach_hoc_bong"},
    {"./gen21052024/Công thức tính điểm ưu tiên.txt": "cong_thuc_tinh_diem_uu_tien"},
    {"./gen21052024/Cẩm nang.txt": "cam_nang"},
    {"./gen21052024/Danh sách các ngành.txt": "danh_sach_nganh"},
    {
        "./gen21052024/Thời gian đăng kí và điều chỉnh nguyện vọng.txt": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong"
    },
    {"./gen21052024/Điểm_cộng_đối_tượng_ưu_tiên.txt": "diem_cong_doi_tuong_uu_tien"},
]

In [43]:
test = list(list_files_[0].items())
key = test[0][0]
value = test[0][1]
key

'./Thông tin tuyển sinh/Chỉ tiêu tuyển sinh.txt'

In [39]:
loader = TextLoader("./Data/" + key)

documents = loader.load()
documents

[Document(page_content='Chỉ tiêu tuyển sinh Học viện Bưu chính Viễn thông năm 2024\nChỉ tiêu tuyển sinh 2024 của Học viện Bưu chính Viễn thông\nTổng chỉ tiêu tuyển sinh 2024 là 5.060 cho cả 2 cơ sở đào tạo tại Hà Nội và Tp. HCM, trong đó chỉ tiêu tuyển sinh của 02 Cơ sở đào tạo như sau:\n1. Chỉ tiêu tuyển sinh tại cơ sở phía Bắc (Mã trường: BVH) tổng chỉ tiêu 3900 gồm 420 chỉ tiêu xét tuyển tài năng, 1950 chỉ tiêu thi THPT (Trung học phổ thông), 965 chỉ tiêu xét tuyển kết hợp (XTKH), 565 chỉ tiêu đánh giá năng lực, đánh giá tư duy (ĐGNL, ĐGTD).\n2. Chỉ tiêu tuyển sinh tại cơ sở phía Nam (Mã trường: BVS) tổng chỉ tiêu 1160 gồm 120 chỉ tiêu xét tuyển tài năng, 575 chỉ tiêu thi THPT (Trung học phổ thông), 280 chỉ tiêu xét tuyển kết hợp (XTKH), 185 chỉ tiêu đánh giá năng lực, đánh giá tư duy (ĐGNL, ĐGTD).\nChi tiết chỉ tiêu tuyển sinh các ngành/nhóm ngành tại các cơ sở theo từng phương thức xét tuyển như sau:\n1. chỉ tiêu tuyển sinh ngành Kỹ thuật Điện tử viễn thông tại cơ sở phía Bắc (BVH

In [52]:
# Khai bao bien
pdf_data_path = list(list_files_[0].items())[0][0]
vector_db_path = "./VectorDB/IntentDetails/"

def create_db_from_files():
    # Khai bao loader de quet toan bo thu muc dataa
    # loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls = PyPDFLoader)
    loader = TextLoader("./Data/" + pdf_data_path)

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
    chunks = text_splitter.split_documents(documents)

    # Embeding
    embed_model = OpenAIEmbeddings()
    db = FAISS.from_documents(chunks, embed_model)
    db.save_local(vector_db_path + value)

    for i in list_files_[1:]:
        i = list(i.items())
        path_load = i[0][0]
        path_save = i[0][1]
        print(path_load)
        loader = TextLoader("./Data/" + path_load)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
        chunks = text_splitter.split_documents(documents)
        db = FAISS.from_documents(chunks, embed_model)
        db.save_local(vector_db_path + path_save)


create_db_from_files()


./Thông tin tuyển sinh/Câu hỏi.txt
./Thông tin tuyển sinh/Học phí.txt
./Thông tin tuyển sinh/Liên hệ và hỗ trợ.txt
./Thông tin tuyển sinh/Phương thức 1.txt
./Thông tin tuyển sinh/Phương thức 2.txt
./Thông tin tuyển sinh/Phương thức 3.txt
./Thông tin tuyển sinh/Phương thức 4.txt
./Thông tin tuyển sinh/Quy đổi chứng chỉ, giải.txt
./Thông tin tuyển sinh/Thông báo mở đăng ký xét tuyển.txt
./Thông tin tuyển sinh/Thông tin tuyển sinh.txt
./Thông tin tuyển sinh/Thời gian đăng kí.txt
./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_kết_hợp_XTKH.txt
./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_ĐGNL,_ĐGTD.txt
./Thông tin tuyển sinh/Tổ hợp xét tuyển.txt
./Thông tin tuyển sinh/Xét học bạ.txt
./test/Các ngành mới.txt
./test/Các ngưỡng đảm bảo đầu vào.txt
./test/Các phương thức xét tuyển.txt
./test/DgnlDgtd.txt
./test/Hồ sơ hợp lệ.txt
./test/Thời gian đăng kí và điều chỉnh nguyện vọng.txt
./test/Địa chỉ nộp hồ sơ xét tuyển.txt
./gen17052024/Câu hỏi.txt
./gen17052024/Giải 

In [ ]:
list_files_ = [
    {"./Thông tin tuyển sinh/Chỉ tiêu tuyển sinh.txt": "chi_tieu_tuyen_sinh"},
    {"./Thông tin tuyển sinh/Câu hỏi.txt": "cau_hoi_xet_tuyen"},
    {"./Thông tin tuyển sinh/Học phí.txt": "hoc_phi"},
    {"./Thông tin tuyển sinh/Liên hệ và hỗ trợ.txt": "thong_tin_lien_he"},
    {"./Thông tin tuyển sinh/Phương thức 1.txt": "phuong_thuc_1"},
    {"./Thông tin tuyển sinh/Phương thức 2.txt": "phuong_thuc_2"},
    {"./Thông tin tuyển sinh/Phương thức 3.txt": "phuong_thuc_3"},
    {"./Thông tin tuyển sinh/Phương thức 4.txt": "phuong_thuc_4"},
    {
        "./Thông tin tuyển sinh/Quy đổi chứng chỉ, giải.txt": "quy_doi_diem_chung_chi_quoc_te"
    },
    {
        "./Thông tin tuyển sinh/Thông báo mở đăng ký xét tuyển.txt": "phuong_thuc_xet_tuyen"
    },
    {"./Thông tin tuyển sinh/Thông tin tuyển sinh.txt": "doi_tuong_tuyen_sinh"},
    {
        "./Thông tin tuyển sinh/Thời gian đăng kí.txt": "thoi_gian_xet_tuyen_va_cong_bo_ket_qua"
    },
    {
        "./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_kết_hợp_XTKH.txt": "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_ket_hop"
    },
    {
        "./Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_ĐGNL,_ĐGTD.txt": "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_danh_gia_nang_luc-tu_duy"
    },
    {"./Thông tin tuyển sinh/Tổ hợp xét tuyển.txt": "to_hop_xet_tuyen"},
    {"./Thông tin tuyển sinh/Xét học bạ.txt": "xet_tuyen_hoc_ba"},
    {"./test/Các ngành mới.txt": "nganh_moi"},
    {"./test/Các ngưỡng đảm bảo đầu vào.txt": "nguong_dam_bao"},
    {"./test/Các phương thức xét tuyển.txt": "phuong_thuc_tuyen_sinh"},
    {"./test/DgnlDgtd.txt": "nguong_dam_bao_danh_gia_nang_luc-tu_duy"},
    {"./test/Hồ sơ hợp lệ.txt": "ho_so_xet_tuyen_hop_le"},
    {
        "./test/Thời gian đăng kí và điều chỉnh nguyện vọng.txt": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong-nop_kinh_phi"
    },
    {"./test/Địa chỉ nộp hồ sơ xét tuyển.txt": "dia_chi_nop_ho_so_xet_tuyen"},
    {"./gen17052024/Câu hỏi.txt": "cau_hoi_xet_tuyen"},
    {"./gen17052024/Giải hsg.txt": "giai_hoc_sinh_gioi"},
    {
        "./gen17052024/Kết thúc đăng ký xét tuyển sớm.txt": "thoi_gian_ket_thuc_dang_ky_xet_tuyen_som"
    },
    {"./gen17052024/Xét học bạ.txt": "xet_tuyen_hoc_ba"},
    {"./gen17052024/Đăng ký xét tuyển.txt": "dang_ky_xet_tuyen"},
    {"./gen21052024/Chính sách học bổng.txt": "chinh_sach_hoc_bong"},
    {"./gen21052024/Công thức tính điểm ưu tiên.txt": "cong_thuc_tinh_diem_uu_tien"},
    {"./gen21052024/Cẩm nang.txt": "cam_nang"},
    {"./gen21052024/Danh sách các ngành.txt": "danh_sach_nganh"},
    {
        "./gen21052024/Thời gian đăng kí và điều chỉnh nguyện vọng.txt": "thoi_gian_dang_ky-dieu_chinh_nguyen_vong"
    },
    {"./gen21052024/Điểm_cộng_đối_tượng_ưu_tiên.txt": "diem_cong_doi_tuong_uu_tien"},
]

In [53]:
list_parent_path = [
    {
        "./Data/thong_tin_tuyen_sinh": [
            "chi_tieu_tuyen_sinh",
            "cau_hoi_xet_tuyen",
            "hoc_phi",
            "thong_tin_lien_he",
            "phuong_thuc_1",
            "phuong_thuc_2",
            "phuong_thuc_3",
            "phuong_thuc_4",
            "quy_doi_diem_chung_chi_quoc_te",
            "phuong_thuc_xet_tuyen",
            "doi_tuong_tuyen_sinh",
            "thoi_gian_xet_tuyen_va_cong_bo_ket_qua",
            "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_ket_hop",
            "thoi_gian_nop_ho_so_dang_ky_xet_tuyen_danh_gia_nang_luc-tu_duy",
            "to_hop_xet_tuyen",
            "xet_tuyen_hoc_ba",
        ]
    },
    {
        "./Data/test": [
            "nganh_moi",
            "nguong_dam_bao",
            "phuong_thuc_tuyen_sinh",
            "nguong_dam_bao_danh_gia_nang_luc-tu_duy",
            "ho_so_xet_tuyen_hop_le",
            "thoi_gian_dang_ky-dieu_chinh_nguyen_vong-nop_kinh_phi",
            "dia_chi_nop_ho_so_xet_tuyen",
        ]
    },
    {
        "./Data/gen17052024": [
            "cau_hoi_xet_tuyen",
            "giai_hoc_sinh_gioi",
            "thoi_gian_ket_thuc_dang_ky_xet_tuyen_som",
            "xet_tuyen_hoc_ba",
            "dang_ky_xet_tuyen",
        ]
    },
    {
        "./Data/gen21052024": [
            "chinh_sach_hoc_bong",
            "cong_thuc_tinh_diem_uu_tien",
            "cam_nang",
            "danh_sach_nganh",
            "thoi_gian_dang_ky-dieu_chinh_nguyen_vong",
            "diem_cong_doi_tuong_uu_tien",
        ]
    },
]

In [58]:
list(list_parent_path[0].items())[0][0]
key = list(list_parent_path[0].items())[0][0]
value = list(list_parent_path[0].items())[0][1]
key, value

('./Data/Thông tin tuyển sinh',
 ['chi_tieu_tuyen_sinh',
  'cau_hoi_xet_tuyen',
  'hoc_phi',
  'thong_tin_lien_he',
  'phuong_thuc_1',
  'phuong_thuc_2',
  'phuong_thuc_3',
  'phuong_thuc_4',
  'quy_doi_diem_chung_chi_quoc_te',
  'phuong_thuc_xet_tuyen',
  'doi_tuong_tuyen_sinh',
  'thoi_gian_xet_tuyen_va_cong_bo_ket_qua',
  'thoi_gian_nop_ho_so_dang_ky_xet_tuyen_ket_hop',
  'thoi_gian_nop_ho_so_dang_ky_xet_tuyen_danh_gia_nang_luc-tu_duy',
  'to_hop_xet_tuyen',
  'xet_tuyen_hoc_ba'])

In [60]:
def remove_vietnamese_accent(s):
    import re
    s = s.lower()
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[đ]', 'd', s)
    s = "_".join(s.split())
    return s

# Khai bao bien
pdf_data_path = list(list_parent_path[0].items())[0][0]
vector_db_path = "./VectorDB/IntentOutlier/"

def create_db_from_files():
    # Khai bao loader de quet toan bo thu muc dataa
    loader = DirectoryLoader(pdf_data_path, glob="*.txt", loader_cls = TextLoader)

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
    chunks = text_splitter.split_documents(documents)

    # Embeding
    embed_model = OpenAIEmbeddings()
    db = FAISS.from_documents(chunks, embed_model)
    path_save = pdf_data_path.split("/")[-1]
    path_save = remove_vietnamese_accent(path_save)
    db.save_local(vector_db_path + path_save)

    for i in list_parent_path[1:]:
        i = list(i.items())
        path_load = i[0][0]
        path_save = path_load.split("/")[-1]
        path_save = remove_vietnamese_accent(path_save)
        print(path_load)
        loader = DirectoryLoader(pdf_data_path, glob="*.txt", loader_cls = TextLoader)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
        chunks = text_splitter.split_documents(documents)
        db = FAISS.from_documents(chunks, embed_model)
        db.save_local(vector_db_path + path_save)


create_db_from_files()


./Data/test
./Data/gen17052024
./Data/gen21052024


In [2]:
def remove_vietnamese_accent(s):
    import re
    s = s.lower()
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[đ]', 'd', s)
    s = "_".join(s.split())
    return s

In [ ]:
import os
import json

list_path_dir = [
    "./Data/test",
    "./Data/Điểm chuẩn PTIT năm 2023",
    "./Data/Thông tin tuyển sinh",
    "./Data/Others",
    './Data/gen17052024',
    './Data/gen21052024',
]

doc_texts = []

doc_check_duplicate = []

for path in list_path_dir:
    for root, dirs, files in os.walk(path):
        for file in files:
            if "txt" in file:
                doc_texts.append(os.path.join(root, file))
                if file not in doc_check_duplicate:
                    doc_check_duplicate.append([0, remove_vietnamese_accent(file), [os.path.join(root, file)]])
                else:
                    for i in doc_check_duplicate:
                        if i[1] == file:
                            i[0] += 1
                            i[2].append(os.path.join(root, file))
                            break
doc_texts

In [ ]:
doc_check_duplicate

# outline

In [14]:
data = [
    {
        "Thông tin trường": [
            "./Data/Others/PTIT-Ban-giam-doc.txt",
            "./Data/Others/PTIT-RIPT.txt",
            "./Data/Others/PTIT.txt",
            "./Data/Others/Thông tin học viện.txt",
        ]
    },
    {
        "Ngành/Chuyên ngành": [
            "./Data/test/Các ngành mới.txt",
            # "./Data/gen21052024/Danh sách các ngành A00 A01 D01.txt",
            # "./Data/gen21052024/Danh sách các ngành A00 A01.txt",
            # "./Data/gen21052024/Danh sách các ngành A00.txt",
            # "./Data/gen21052024/Danh sách các ngành A01.txt",
            # "./Data/gen21052024/Danh sách các ngành D01.txt",
            "./Data/gen21052024/Danh sách các ngành.txt",
        ]
    },
    {
        "Ban tuyển sinh": [
            "./Data/Thông tin tuyển sinh/Liên hệ và hỗ trợ.txt",
            "./Data/test/Địa chỉ nộp hồ sơ xét tuyển.txt",
        ]
    },
    {
        "Quy đổi chứng chỉ": [
            "./Data/Thông tin tuyển sinh/Quy đổi chứng chỉ, giải.txt",
        ]
    },
    {
        "Xét tuyển thẳng": [
            "./Data/Thông tin tuyển sinh/Thành viên Olympic.txt",
        ]
    },
    {
        "Phương thức 1": [
            "./Data/Thông tin tuyển sinh/Phương thức 1.txt",
            "./Data/Thông tin tuyển sinh/Học sinh chuyên.txt",
            "./Data/Thông tin tuyển sinh/Có giải học sinh giỏi.txt",
            "./Data/Thông tin tuyển sinh/Ưu tiên giải học sinh giỏi.txt",
        ]
    },
    {
        "Phương thức 2": [
            "./Data/Thông tin tuyển sinh/Phương thức 2.txt",
        ]
    },
    {
        "Phương thức 3": [
            "./Data/Thông tin tuyển sinh/Phương thức 3.txt",
            "./Data/Thông tin tuyển sinh/Có chứng chỉ.txt",
        ]
    },
    {
        "Phương thức 4": [
            "./Data/Thông tin tuyển sinh/Phương thức 4.txt",
            "./Data/test/DgnlDgtd.txt",
        ]
    },
    {
        "Phương thức xét tuyển": [
            "./Data/gen21052024/Cẩm nang.txt",
            "./Data/gen21052024/Công thức tính điểm ưu tiên.txt",
            "./Data/Thông tin tuyển sinh/Mix giải chứng chỉ chuyên.txt",
            "./Data/Thông tin tuyển sinh/Thông tin tuyển sinh.txt",
            "./Data/Thông tin tuyển sinh/Tổ hợp xét tuyển.txt",
            "./Data/Thông tin tuyển sinh/Đăng ký xét tuyển.txt",
            "./Data/gen17052024/Câu hỏi.txt",
            "./Data/gen17052024/Đăng ký xét tuyển.txt",
        ]
    },
    {"Thời gian": [
            "./Data/Thông tin tuyển sinh/Thông báo mở đăng ký xét tuyển.txt",
            "./Data/Thông tin tuyển sinh/Thời gian đăng kí.txt",
            "./Data/Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_kết_hợp_XTKH.txt",
            "./Data/Thông tin tuyển sinh/Thời_gian_nộp_hồ_sơ_đăng_ký_xét_tuyển_ĐGNL,_ĐGTD.txt",
            "./Data/gen17052024/Kết thúc đăng ký xét tuyển sớm.txt",
            "./Data/gen21052024/Thời gian đăng kí và điều chỉnh nguyện vọng.txt",
            "./Data/test/Thời gian nộp kinh phí xét tuyển.txt",
            "./Data/test/Thời gian đăng kí và điều chỉnh nguyện vọng.txt",
            "./Data/Thông tin tuyển sinh/Kết thúc đăng ký xét tuyển sớm.txt",

    ]},
    {"Hồ sơ": [
            "./Data/Thông tin tuyển sinh/Hồ sơ phương thức 1.txt",
            "./Data/Thông tin tuyển sinh/Hồ sơ phương thức 3.txt",
            "./Data/Thông tin tuyển sinh/Hồ sơ phương thức 4.txt",
            "./Data/test/Hồ sơ hợp lệ.txt",
            "./Data/test/Địa chỉ nộp hồ sơ xét tuyển.txt",

    ]},
    {
        "Chỉ tiêu tuyển sinh": [
            "./Data/Thông tin tuyển sinh/Chỉ tiêu tuyển sinh.txt",
        ]
    },
    {
        "Điểm trúng tuyển": [
            "./Data/Điểm chuẩn PTIT năm 2023/Tổng hợp điểm chuẩn PTIT 2023.txt",
        ]
    },
    {
        "Đối tượng ưu tiên, khu vực ưu tiên": [
            "./Data/test/Đối_tượng_ưu_tiên.txt",
            "./Data/gen21052024/Điểm_cộng_khu_vực_ưu_tiên.txt",
            "./Data/gen21052024/Điểm_cộng_đối_tượng_ưu_tiên.txt",
            "./Data/gen21052024/Khu_vực_ưu_tiên.txt",
        ]
    },
    {
        "Học phí": [
            "./Data/Thông tin tuyển sinh/Học phí.txt",
        ]
    },
    {
        "Chính sách hỗ trợ, ưu tiên": [
            "./Data/Thông tin tuyển sinh/Chính sách ưu tiên.txt",
        ]
    },
    {
        "Học bổng": [
            "./Data/gen21052024/Chính sách học bổng.txt",
        ]
    },
    {
        "Cơ hội việc làm": [
            "./Data/Others/Tình hình việc làm.txt",
        ]
    },
    {
        "Hợp tác đối ngoại": [
            "./Data/Others/Doanh nghiệp hợp tác.txt",
        ]
    },
    {
        "Cách thức xét tuyển các phương thức": [
            "./Data/test/Các ngưỡng đảm bảo đầu vào.txt",
            "./Data/test/Các phương thức xét tuyển.txt",
        ]
    },
    {
        "Giải học sinh giỏi": [
            "./Data/Thông tin tuyển sinh/Có giải học sinh giỏi.txt",
            "./Data/Thông tin tuyển sinh/Ưu tiên giải học sinh giỏi.txt",
        ]
    },
    {
        "Xét học bạ": [
            "./Data/gen17052024/Xét học bạ.txt",
        ]
    },
    {
        "Khác": [
            "./Data/Others/FTU.txt",
            "./Data/Others/HaUI.txt",
            "./Data/Others/HCMUS.txt",
            "./Data/Others/HUST.txt",
            "./Data/Others/NEU.txt",
            "./Data/Others/UDU.txt",
            "./Data/Others/VNU-UET.txt",
            "./Data/Phòng đào tạo/Hoc_phi.txt",
            "./Data/Phòng đào tạo/Nganh_daotao.txt",
            "./Data/Phòng đào tạo/Phuong_thuc_tuyen_sinh.txt",
            "./Data/Phòng đào tạo/Thuong_gap.txt",
            "./Data/Phòng đào tạo/Ưu tiên.txt",
        ]
    },
]

In [21]:
import os

list_json = os.listdir("./Data/gen_intent/")
list_json = [i.split(".")[0] for i in list_json]
list_check_gen = []

In [22]:
for i in data:
    for key, value in i.items():
        index = data.index(i)
        # print(data[index][key])
        # print(i[key])
        # print(key)
        for v in value:
            # print(value.index(v), v)
            v_remove = v.split("\\")[-1].split(".")[0]
            v_object = {
                v: remove_vietnamese_accent(v_remove)
            }
            data[index][key][value.index(v)] = v_object
            if v_object not in list_check_gen:
                list_check_gen.append(remove_vietnamese_accent(v_remove))

In [23]:
for i in list_check_gen:
    if i not in list_json:
        print(i)

diem_cong_khu_vuc_uu_tien
khu_vuc_uu_tien
chinh_sach_uu_tien
nganh_daotao
phuong_thuc_tuyen_sinh
thuong_gap
uu_tien


In [10]:
def remove_vietnamese_accent(s):
    import re
    s = s.lower()
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[đ]', 'd', s)
    s = "_".join(s.split())
    return s

In [26]:
dict_data = {}
import json

for i in list_json:
    if os.path.exists(f"./Data/gen_intent/{i}.json"):
        with open(f"./Data/gen_intent/{i}.json", "r", encoding="utf-8") as f:
            data = json.load(f)
            for k, q in data["queries"].items():
                if i not in dict_data:
                    dict_data[i] = []
                dict_data[i].append(q)
    # for key, value in i.items():
    #     with open(f"{key}.json", "r", encoding="utf-8") as f:
    #         data = json.load(f)
    #         # print(value)
    #         if value not in dict_data:
    #             dict_data[value] = []
    #         for k, q in data['queries'].items():
    #             dict_data[value].append(q)
dict_data

{'cac_nganh_moi': ['Ngành/chương trình mới nào sẽ được tuyển sinh từ năm 2024 theo thông báo của học viện?',
  'Ngành Thiết kế và Phát triển Game sẽ được tuyển sinh từ năm nào?',
  'Ngành/chương trình nào dự kiến sẽ có thông báo bổ sung vào tháng 5/2024?',
  'Ngành/chương trình nào sẽ được đào tạo tại Học viện?',
  'Ngành/chương trình nào sẽ được đào tạo theo hợp tác giữa Việt Nam và Nhật Bản?',
  'Ngành/chương trình nào sẽ tập trung vào lĩnh vực Công nghệ thông tin?',
  'Ngành/chương trình nào sẽ tập trung vào lĩnh vực Quan hệ công chúng?',
  'Khi nào các ngành mới dự kiến sẽ bắt đầu tuyển sinh?',
  'Có bao nhiêu ngành/chương trình mới sẽ được tuyển sinh từ năm 2024?',
  'Ngành/chương trình nào sẽ cung cấp kiến thức về Thiết kế và Phát triển Game?',
  'Ngành/chương trình nào sẽ cung cấp kiến thức về Công nghệ thông tin?',
  'Ngành/chương trình nào sẽ cung cấp kiến thức về Quan hệ công chúng?',
  'Ngành/chương trình nào sẽ hợp tác giữa Việt Nam và Nhật Bản?',
  'Ngành/chương trình nào 

In [29]:
# code
import json
import re
""" 
Xóa dấu tiếng việt của intent
ex: địa chỉ cơ sở -> dia_chi_co_so
 """

def remove_vietnamese_accent(s):
    s = s.lower()
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[đ]', 'd', s)
    s = "_".join(s.split())
    return s

# xóa kí tự đặc biệt 
import re
import string

# ex: #x009f
def remove_unacceptable_characters(text):
  return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

with open('./nlu.yml', 'w', encoding='utf-8') as f:
    f.write(f'version: "3.1"\n\nnlu:\n')
    for i in data:
        for key, value in i.items():
            intent = remove_vietnamese_accent(key)
            f.write(f'- intent: {intent}\n')
            f.write(f'  examples: |\n')
            for v in value:
                v = v.split("\\")[-1].split(".")[0]
                v = remove_vietnamese_accent(v)
                if v in dict_data:
                    for j in dict_data[v]:
                        f.write(f'    - {remove_unacceptable_characters(j)}\n')
            f.write('\n')
                
            
        


In [30]:
list_key = [list(i.keys())[0] for i in data]
list_key

['Thông tin trường',
 'Ngành/Chuyên ngành',
 'Ban tuyển sinh',
 'Quy đổi chứng chỉ',
 'Xét tuyển thẳng',
 'Phương thức 1',
 'Phương thức 2',
 'Phương thức 3',
 'Phương thức 4',
 'Phương thức xét tuyển',
 'Thời gian',
 'Hồ sơ',
 'Chỉ tiêu tuyển sinh',
 'Điểm trúng tuyển',
 'Đối tượng ưu tiên, khu vực ưu tiên',
 'Học phí',
 'Chính sách hỗ trợ, ưu tiên',
 'Học bổng',
 'Cơ hội việc làm',
 'Hợp tác đối ngoại',
 'Cách thức xét tuyển các phương thức',
 'Giải học sinh giỏi',
 'Xét học bạ',
 'Khác']

In [34]:
pdf_data_path = []
for i in data:
    pdf_data_path.append(list(i.items())[0][0])

# ingestion vectorDB

In [17]:
# Khai bao bien
from langchain_community.vectorstores.faiss import DistanceStrategy

vector_db_path = "./VectorDB/IntentOutline/"

def create_db_from_files():
    # Khai bao loader de quet toan bo thu muc dataa
    # loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls = PyPDFLoader)

    # Embeding
    embed_model = OpenAIEmbeddings()
    # db = FAISS.from_documents(chunks, embed_model, distance_strategy = DistanceStrategy.COSINE)

    for i in data:
        i = list(i.items())
        path_save = i[0][0]
        path_save = remove_vietnamese_accent(path_save)
        path_load = i[0][1]
        print(path_load, path_save)
        loader = TextLoader(path_load[0])
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
        chunks = text_splitter.split_documents(documents)
        print(chunks)
        
        if len(path_load) > 1:
            for p in path_load[1:]:
                loader = TextLoader(p)
                documents = loader.load()
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
                chunks_item = text_splitter.split_documents(documents)
                chunks = chunks + chunks_item
                
        db = FAISS.from_documents(chunks, embed_model, distance_strategy = DistanceStrategy.COSINE)
        db.save_local(vector_db_path + path_save)
            
        # print(path_load)
        # loader = TextLoader("./Data/" + path_load)
        # documents = loader.load()
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=2512, chunk_overlap=256)
        # chunks = text_splitter.split_documents(documents)
        # db = FAISS.from_documents(chunks, embed_model)
        # db.save_local(vector_db_path + path_save)


create_db_from_files()

['./Data/Others/PTIT-Ban-giam-doc.txt', './Data/Others/PTIT-RIPT.txt', './Data/Others/PTIT.txt', './Data/Others/Thông tin học viện.txt'] thong_tin_truong
[Document(page_content='BAN GIÁM ĐỐC HỌC VIỆN CÔNG NGHỆ BƯU CHÍNH VIỄN THÔNG\n\nGiám đốc Học viện PGS.TS. Đặng Hoài Bắc\nPhó Giám đốc Học viện PGS.TS. Trần Quang Anh\nPhó Giám đốc Học viện TS. Tân Hạnh', metadata={'source': './Data/Others/PTIT-Ban-giam-doc.txt'})]
['./Data/test/Các ngành mới.txt', './Data/gen21052024/Danh sách các ngành.txt'] nganh/chuyen_nganh
[Document(page_content='Học viện thông báo sẽ tuyển sinh các ngành/chương trình mới sau:\nCác ngành mới, chương trình đào tạo mới dự kiến sẽ tuyển sinh từ năm 2024 gồm có Thiết kế và Phát triển Game, Công nghệ thông tin Việt – Nhật, Quan hệ công chúng (sẽ có thông báo bổ sung dự kiến vào tháng 5/2024).', metadata={'source': './Data/test/Các ngành mới.txt'})]
['./Data/Thông tin tuyển sinh/Liên hệ và hỗ trợ.txt', './Data/test/Địa chỉ nộp hồ sơ xét tuyển.txt'] ban_tuyen_sinh
[Docu

In [34]:
import os

list_intent = os.walk("./VectorDB/IntentOutline/")
# list_intent = [i.split(".")[0] for i in list_intent]
for root, dirs, files in list_intent:
    print(root, dirs, files)

./VectorDB/IntentOutline/ ['ban_tuyen_sinh', 'cach_thuc_xet_tuyen_cac_phuong_thuc', 'chinh_sach_ho_tro,_uu_tien', 'chi_tieu_tuyen_sinh', 'co_hoi_viec_lam', 'diem_trung_tuyen', 'doi_tuong_uu_tien,_khu_vuc_uu_tien', 'giai_hoc_sinh_gioi', 'hoc_bong', 'hoc_phi', 'hop_tac_doi_ngoai', 'ho_so', 'khac', 'nganh', 'phuong_thuc_1', 'phuong_thuc_2', 'phuong_thuc_3', 'phuong_thuc_4', 'phuong_thuc_xet_tuyen', 'quy_doi_chung_chi', 'thoi_gian', 'thong_tin_truong', 'xet_hoc_ba', 'xet_tuyen_thang'] []
./VectorDB/IntentOutline/ban_tuyen_sinh [] ['index.faiss', 'index.pkl']
./VectorDB/IntentOutline/cach_thuc_xet_tuyen_cac_phuong_thuc [] ['index.faiss', 'index.pkl']
./VectorDB/IntentOutline/chinh_sach_ho_tro,_uu_tien [] ['index.faiss', 'index.pkl']
./VectorDB/IntentOutline/chi_tieu_tuyen_sinh [] ['index.faiss', 'index.pkl']
./VectorDB/IntentOutline/co_hoi_viec_lam [] ['index.faiss', 'index.pkl']
./VectorDB/IntentOutline/diem_trung_tuyen [] ['index.faiss', 'index.pkl']
./VectorDB/IntentOutline/doi_tuong_uu_

# Load vectorDB

In [19]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-tZ5Uxrp7mtytZGbGRAMWT3BlbkFJIMLK05g3ZOlYdz8PIHkK"

In [4]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

dict_vectorstore = {
    "thoi_gian": "./VectorDB/IntentOutline/thoi_gian",
    "ho_so": "./VectorDB/IntentOutline/ho_so",
    "ban_tuyen_sinh": "./VectorDB/IntentOutline/ban_tuyen_sinh",
    "cach_thuc_xet_tuyen_cac_phuong_thuc": "./VectorDB/IntentOutline/cach_thuc_xet_tuyen_cac_phuong_thuc",
    "chinh_sach_ho_tro,_uu_tien": "./VectorDB/IntentOutline/chinh_sach_ho_tro,_uu_tien",
    "chi_tieu_tuyen_sinh": "./VectorDB/IntentOutline/chi_tieu_tuyen_sinh",
    "co_hoi_viec_lam": "./VectorDB/IntentOutline/co_hoi_viec_lam",
    "diem_trung_tuyen": "./VectorDB/IntentOutline/diem_trung_tuyen",
    "doi_tuong_uu_tien,_khu_vuc_uu_tien": "./VectorDB/IntentOutline/doi_tuong_uu_tien,_khu_vuc_uu_tien",
    "giai_hoc_sinh_gioi": "./VectorDB/IntentOutline/giai_hoc_sinh_gioi",
    "hoc_bong": "./VectorDB/IntentOutline/hoc_bong",
    "hoc_phi": "./VectorDB/IntentOutline/hoc_phi",
    "hop_tac_doi_ngoai": "./VectorDB/IntentOutline/hop_tac_doi_ngoai",
    "khac": "./VectorDB/IntentOutline/khac",
    "nganh\chuyen_nganh": "./VectorDB/IntentOutline/nganh\chuyen_nganh",
    "phuong_thuc_1": "./VectorDB/IntentOutline/phuong_thuc_1",
    "phuong_thuc_2": "./VectorDB/IntentOutline/phuong_thuc_2",
    "phuong_thuc_3": "./VectorDB/IntentOutline/phuong_thuc_3",
    "phuong_thuc_4": "./VectorDB/IntentOutline/phuong_thuc_4",
    "phuong_thuc_xet_tuyen": "./VectorDB/IntentOutline/phuong_thuc_xet_tuyen",
    "quy_doi_chung_chi": "./VectorDB/IntentOutline/quy_doi_chung_chi",
    "thong_tin_truong": "./VectorDB/IntentOutline/thong_tin_truong",
    "xet_hoc_ba": "./VectorDB/IntentOutline/xet_hoc_ba",
    "xet_tuyen_thang": "./VectorDB/IntentOutline/xet_tuyen_thang",
}

for key, value in dict_vectorstore.items():
    vectorstore = FAISS.load_local(
        value,
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True,
    )
    dict_vectorstore[key] = vectorstore.as_retriever(
        search_kwargs={"k": 3, "threshold": 0.5}
    )

dict_vectorstore

{'ban_tuyen_sinh': VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002098E239250>, search_kwargs={'k': 3, 'threshold': 0.5}),
 'cach_thuc_xet_tuyen_cac_phuong_thuc': VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002098FC41D90>, search_kwargs={'k': 3, 'threshold': 0.5}),
 'chinh_sach_ho_tro,_uu_tien': VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002098FD622D0>, search_kwargs={'k': 3, 'threshold': 0.5}),
 'chi_tieu_tuyen_sinh': VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002098FD74C50>, search_kwargs={'k': 3, 'threshold': 0.5}),
 'co_hoi_viec_lam': VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.F

In [13]:
question = "thời gian nộp hồ sơ xét tuyển kết hợp"

In [16]:
import requests

def get_intent(text):
    url = "http://localhost:5005/model/parse"
    payload = {
        "text": text
    }
    response = requests.post(url, json=payload)
    return response.json()

response = get_intent(question)
response['intent_ranking']

[{'name': 'nlu_fallback', 'confidence': 0.7},
 {'name': 'phuong_thuc_3', 'confidence': 0.6950944662094116},
 {'name': 'dieu_kien_quy_trinh_phuong_thuc_3',
  'confidence': 0.22888830304145813},
 {'name': 'phuong_thuc_2', 'confidence': 0.016751935705542564},
 {'name': 'xet_tuyen_hoc_ba', 'confidence': 0.010074673220515251},
 {'name': 'thoi_gian_nop_ho_so_dang_ky_xet_tuyen_ket_hop',
  'confidence': 0.0073378803208470345},
 {'name': 'thoi_gian_xet_tuyen_va_cong_bo_ket_qua',
  'confidence': 0.0033613997511565685},
 {'name': 'thoi_gian_nop_ho_so_dang_ky_xet_tuyen_danh_gia_nang_luc-tu_duy',
  'confidence': 0.0032710202503949404},
 {'name': 'thong_tin_lien_he', 'confidence': 0.002842770889401436},
 {'name': 'phuong_thuc_tuyen_sinh', 'confidence': 0.0022526741959154606},
 {'name': 'thoi_gian_dang_ky-dieu_chinh_nguyen_vong-nop_kinh_phi',
  'confidence': 0.0019149527652189136}]

In [17]:
import requests

def get_intent(text):
    url = "http://localhost:5005/model/parse"
    payload = {
        "text": text
    }
    response = requests.post(url, json=payload)
    return response.json()

response = get_intent(question)
response['intent_ranking']
docs = []
# response = get_intent(question)
# response['intent_ranking']

top_5_intent = response['intent_ranking'][:5]
print(top_5_intent)

list_docs = []
for i in top_5_intent:
    if "nlu_fallback" in i['name']:
        continue
    else:
        intent = i['name']
        for key, value in dict_vectorstore.items():
            if intent == key:
                print(intent, key)
                vectorstore = dict_vectorstore[key]
                docs1 = vectorstore.get_relevant_documents(question)
                list_docs.append(docs1)

[{'name': 'nlu_fallback', 'confidence': 0.7}, {'name': 'phuong_thuc_3', 'confidence': 0.6950944662094116}, {'name': 'dieu_kien_quy_trinh_phuong_thuc_3', 'confidence': 0.22888830304145813}, {'name': 'phuong_thuc_2', 'confidence': 0.016751935705542564}, {'name': 'xet_tuyen_hoc_ba', 'confidence': 0.010074673220515251}]
phuong_thuc_3 phuong_thuc_3
phuong_thuc_2 phuong_thuc_2


In [18]:
list_docs

[[Document(page_content='Thời gian nộp hồ sơ đăng ký xét tuyển phương thức 3:\nThời gian nộp hồ sơ đăng ký xét tuyển kết hợp (XTKH):\n- Từ ngày 15/4/2024, thí sinh đăng ký tài khoản xét tuyển trên hệ thống xết tuyển trực tuyến của Học viện.\n- Từ ngày 15/4 đến 0h00 ngày 26/5/2024, thí sinh đăng ký, điều chỉnh nguyện vọng xét tuyển (không giới hạn số lần) trên Hệ thống xét tuyển trực tuyến của Học viện.\n- Từ ngày 15/4 đến 0h00 ngày 26/5/2024, thí sinh nộp kinh phí xét tuyển (trực tuyến bằng mã định danh hoặc mã QR).\n- Từ ngày 15/4 đến 0h00 ngày 26/5/2024, thí sinh rà xoát và khóa hồ sơ đăng ký xét tuyển trên Hệ thống xét tuyển của Học viện.\n- Từ ngày 15/4 đến 17h00 ngày 25/5/2024, thí sinh nộp hồ sơ đăng ký xét tuyển theo quy định của Học viện qua đường bưu điện chuyển phát nhanh hoặc chuyển phát đảm bảo.\n- Dự kiến từ ngày 10/6 đến ngày 15/6/2024, Học viện thông báo thí sinh đủ điều kiện trúng tuyển.\n- Từ ngày 10/7 đến 17h00 ngày 25/07/20024, thí sinh đủ điều kiện trúng tuyển đăng 

In [3]:
dict_replace = {
    "2k|2k|2k|2k|2k": "200",
    "a": "anh",
    "ac": "anh chị",
    "a/c": "anh chị",
    "add|ad|adm|addd": "admin",
    "ak|ah": "à",
    "b": "bạn",
    "baoh|bh": "bao giờ",
    "bhyt": "bảo hiểm y tế",
    "bn|bnh|baonh": "bao nhiêu",
    "bâyh": "bây giờ",
    "cccd": "căn cước công dân",
    "chòa": "chào",
    "cj": "chị",
    "ck": "chuyển khoản",
    "chỉ tiêu": "chỉ tiêu tuyển sinh",
    "chỉ tiêu tuyển sinh tuyển sinh": "chỉ tiêu tuyển sinh",
    "clc": "chất lượng cao",
    "cmnd": "chứng minh nhân dân",
    "dgnl": "đánh giá năng lực",
    "dgtd": "đánh giá tư duy",
    "dhqg": "đại học quốc gia",
    "dkxt": "đăng ký xét tuyển",
    "dko": "được không",
    "đko": "được không",
    "dky|dki|đăng kí|dki|đki": "đăng ký",
    "đkxt|dkxt": "đăng ký xét tuyển",
    "hvcnbtvt": "Học viện Công nghệ Bưu chính Viễn thông",
    "d|đ": "điểm",
    "e": "em",
    "gddt|gd dt|gdđt|gd đt": "giáo dục đào tạo",
    "gd": "giáo dục",
    "hc": "học",
    "hcm": "hồ chí minh",
    "hnay": "hôm nay",
    "hqua": "hôm qua",
    "điểm tuyển sinh": "điểm trúng tuyển",
    "điểm đỗ": "điểm trúng tuyển",
    "điểm chuẩn|điểm": "điểm trúng tuyển",
    "điểm trúng tuyển đỗ": "điểm trúng tuyển",
    "điểm trúng tuyển điểm trúng tuyển": "điểm trúng tuyển",
    "điểm trúng tuyển trúng tuyển": "điểm trúng tuyển",
    "điểm trúng tuyển tuyển sinh": "điểm trúng tuyển",
    "điểm trúng tuyển": "điểm trúng tuyển",
    "điểm trúng tuyển nổi bật": "điểm nổi bật",
    "điểm trúng tuyển ưu tiên": "điểm ưu tiên",
    "điểm trúng tuyển cộng": "điểm cộng",
    "cộng điểm trúng tuyển": "cộng điểm",
    "điểm trúng tuyển thưởng": "điểm thưởng",
    "hv": "học viện",
    "j": "gì",
    "ko|k|kh|khg|kg|hong|hok|khum": "không",
    "ktx": "ký túc xá",
    "kt": "Kế toán",
    "kv|khvuc|kvuc": "khu vực",
    "lm": "làm",
    "lpxt": "lệ phí xét tuyển",
    "m": "mình",
    "mn": "mọi người",
    "tkdh": "thiết kế đồ họa",
    "mk": "mình",
    "nnao": "như nào",
    "ntn": "như thế nào",
    "nv|nvong": "nguyện vọng",
    "oke|oki|okeee": "ok",
    "p": "phải",
    "pk": "phải không",
    "q1|q9": "quận",
    "r": "rồi",
    "sdt|sđt|dt|đt": "số điện thoại",
    "sv": "sinh viên",
    "t": "tôi",
    "thpt": "trung học phổ thông",
    "thptqg": "trung học phổ thông quốc gia",
    "tk": "tài khoản",
    "tmdt": "Thương mại điện tử",
    "tnthpt": "trắc nghiệm trung học phổ thông",
    "tp hcm": "thành phố hồ chí minh",
    "trg|tr|trườg": "trường",
    "trường": "học viện",
    "ttnv": "thứ thự nguyện vọng",
    "tt": "thông tin",
    "uh|uk|um": "ừ",
    "vc": "việc",
    "vd": "ví dụ",
    "vs": "với",
    "z|v": "vậy",
    "đc|dc": "được",
    "năm nay": "2024",
    "năm ngoái|năm trước": "2023",
    "A1|a1": "A01",
    "A0|a0": "A00",
    "D1|d1": "D01",
    "A01": "tổ hợp xét tuyển A01",
    "A00": "tổ hợp xét tuyển A00",
    "D01": "tổ hợp xét tuyển D01",
    "hsg": "học sinh giỏi",
    "giải 1": "giải nhất học sinh giỏi",
    "giải nhất học sinh giỏi học sinh giỏi": "giải nhất học sinh giỏi",
    "giải 2": "giải nhì học sinh giỏi",
    "giải nhì học sinh giỏi học sinh giỏi": "giải nhì học sinh giỏi",
    "giải 3": "giải ba học sinh giỏi",
    "giải ba học sinh giỏi học sinh giỏi": "giải ba học sinh giỏi",
    "tổ hợp": "tổ hợp xét tuyển",
    "khối": "tổ hợp xét tuyển",
    "tổ hợp xét tuyển tổ hợp xét tuyển": "tổ hợp xét tuyển",
    "hsnl": "hồ sơ năng lực",
    "hthuc": "hình thức",
    "hthức": "hình thức",
    "hso": "hồ sơ",
    "hs": "hồ sơ",
    "nl": "năng lực",
    "hssv": "hồ sơ sinh viên",
    "xttn": "xét tuyển tài năng",
    "xtkh|xthk": "xét tuyển kết hợp",
    "xt": "xét tuyển",
    "xtkn": "xét tuyển kết hợp",
    "hba": "học bạ",
    "rv": "review",
    "review": "thông tin",
    "pthuc": "phương thức",
    "pt": "phương thức",
    "ielts": "IELTS",
    "toefl": "TOEFL",
    "toeic": "TOEIC",
    "sat": "SAT",
    "act": "ACT",
    "toefl itp": "TOEFL ITP",
    "toefl ibt": "TOEFL iBT",
    "xét": "xét tuyển",
    "xét tuyển tuyển": "xét tuyển",
    "Tổ hợp xét tuyển": "tổ hợp xét tuyển",
    "ktdtvt": "kỹ thuật điện tử viễn thông",
    "dtvt|dt vt|đtvt|đt vt": "điện tử viễn thông",
    "vt": "Viễn thông",
    "attt": "An toàn thông tin",
    "ttdpt|ttđpt": "Truyền thông đa phương tiện",
    "cn dpt|cndpt": "Công nghệ đa phương tiện",
    "udu": "Công nghệ thông tin định hướng ứng dụng",
    "cntt udu": "Công nghệ thông tin định hướng ứng dụng",
    "cntt ứng dụng": "Công nghệ thông tin định hướng ứng dụng",
    "công nghệ thông tin ứng dụng": "Công nghệ thông tin định hướng ứng dụng",
    "cntt": "Công nghệ thông tin",
    "dpt|đpt": "đa phương tiện",
    "khmt": "Khoa học máy tính",
    "ktdk|ktđk": "Kỹ thuật điều khiển",
    "ktdktdh": "Kỹ thuật điều khiển và tự động hóa",
    "iot": "Công nghệ Internet vạn vật (IoT)",
    "qtkd": "Quản trị kinh doanh",
    "ngành kt": "Ngành kế toán",
    "mmt": "Mạng máy tính",
    "bc": "Báo chí",
    "cntc": "Công nghệ tài chính (Fintech)",
    "marketting|makettting|mkt": "marketing",
}


def normalize_replace_abbreviation_text(text):
    import re
    # text = re.sub(
    #     r"[\.,\(\)]", " ", text
    # )  # thay thế các kí tự đặc biệt bằng khoảng trắng
    # text = re.sub("<.*?>", "", text).strip()
    # text = re.sub("(\s)+", r"\1", text)
    # chars = re.escape(string.punctuation)
    # text = re.sub(
    #     r"[" + chars + "]", " ", text
    # )  # thay thế các kí tự đặc biệt bằng khoảng trắng
    text = re.sub(r"\s+", " ", text)  # thay thế nhiều khoảng trắng bằng 1 khoảng trắng
    text = text.strip()  # xóa khoảng trắng ở đầu và cuối
    text = text.lower()  # chuyển về chữ thường
    """ 
    # "cntt" -> "công nghệ thông tin"
    text = re.sub(r'\bcntt\b', 'công nghệ thông tin', text)
    # "ntn" -> "như thế nào"
    text = re.sub(r'\bntn\b', 'như thế nào', text)
    # "ad, adm" -> "admin"
    text = re.sub(r'\b(ad|adm)\b', 'admin', text)
    text = re.sub(r'\b(gd dt|gddt)\b', 'giáo dục đào tạo', text) 
    # điểm chuẩn -> điểm trúng tuyển
    text = re.sub(r'\bđiểm chuẩn\b', 'điểm trúng tuyển', text)
    """

    for k, v in dict_replace.items():
        text = re.sub(r"\b" + "(" + k + ")" + r"\b", v, text)

    return text.lower()

In [5]:
text = normalize_replace_abbreviation_text("e được giải nhất môn toán thì có đc cộng điểm ko")
text

'em được giải nhất môn toán thì có được cộng điểm không'